### Materials

In [1]:
import pandas
import numpy as np
import os
import shutil
import torch

In [7]:
labels = ["can","bottle"]
for label in labels:
    os.makedirs(f'material_classification/train/{label}')
    os.makedirs(f'material_classification/val/{label}')
    os.makedirs(f'material_classification/test/{label}')

In [8]:
bottle_train_count = 0
bottle_val_count = 0
bottle_test_count = 0
can_train_count = 0
can_val_count = 0
can_test_count = 0

sub_dir = ["coca cola can", "coca cola plastic bottle", "fanta can", 
           "fanta plastic bottle", "7up can", "7up plastic bottle"]
for d in sub_dir:
    if d.split()[-1] == "bottle":
        for f in os.listdir(f'DATA/data/{d}'):
            if bottle_train_count < 2400:
                shutil.copy(f'DATA/data/{d}/'+f,f'material_classification/train/bottle/{bottle_train_count}.jpg')
                bottle_train_count += 1
            elif bottle_val_count < 300:
                shutil.copy(f'DATA/data/{d}/'+f,f'material_classification/val/bottle/{bottle_val_count}.jpg')
                bottle_val_count += 1
            elif bottle_test_count < 300:
                shutil.copy(f'DATA/data/{d}/'+f,f'material_classification/test/bottle/{bottle_test_count}.jpg')
                bottle_test_count += 1
    elif d.split()[-1] == "can":
        for f in os.listdir(f'DATA/data/{d}'):
            if can_train_count < 2400:
                shutil.copy(f'DATA/data/{d}/'+f,f'material_classification/train/can/{can_train_count}.jpg')
                can_train_count += 1
            elif can_val_count < 300:
                shutil.copy(f'DATA/data/{d}/'+f,f'material_classification/val/can/{can_val_count}.jpg')
                can_val_count += 1
            elif can_test_count < 300:
                shutil.copy(f'DATA/data/{d}/'+f,f'material_classification/test/can/{can_test_count}.jpg')
                can_test_count += 1

In [2]:
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F
import torchvision
from torchvision import models, transforms, datasets
from collections import OrderedDict

In [3]:
dataset_transform = transforms.Compose([
  transforms.Resize(256),
  transforms.CenterCrop(224),
  transforms.RandomRotation(30),
  transforms.RandomHorizontalFlip(),
  transforms.ToTensor(),
  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [4]:
train_dir = 'material_classification/train'
val_dir = 'material_classification/val'
test_dir = 'material_classification/test'

In [5]:
# Create train, validation and test dataset
train_dataset = datasets.ImageFolder(train_dir,transform=dataset_transform)
val_dataset = datasets.ImageFolder(val_dir,transform=dataset_transform)
test_dataset = datasets.ImageFolder(test_dir,transform=dataset_transform)

In [6]:
# Load datasets into dataloader for iteration
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = 100, shuffle = True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size = 100, shuffle = True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = 100, shuffle = True)

In [7]:
# Training function
def training(model):
    iteration = 0
    minLoss = float("inf")
    best_parameters = None

    # Training Loop
    for epoch in range(epochs):
        # Iterate over the training dataloader
        for i, (images, labels) in enumerate(train_dataloader):
            # Reinitialize the gradient
            optimizer.zero_grad()

            # Get the correct label
            images, labels = images.to(device), labels.to(device)

            # Forward propagation
            outputs = model(images)

            # Calculate the loss
            loss = criterion(outputs, labels)

            # Use the loss for back propagation
            loss.backward()

            # Update network parameters
            optimizer.step()
            
            # Empty the cache of GPU
            del images
            del labels
            del loss
            del outputs
            torch.cuda.empty_cache()

            iteration += 1

            if iteration % 10 == 0 or iteration == 1:
                # Calculate Accuracy and loss        
                correct = 0
                total = 0

                total_loss = 0

                # Validation Loop
                # Iterate over the validation dataloader
                for images, labels in val_dataloader:

                    images, labels = images.to(device), labels.to(device)
                    # Make prediction
                    outputs = model(images)

                    loss = criterion(outputs, labels)
                    total_loss += float(loss)

                    # Get predictions from the maximum value, index 0 is the maximum value, 
                    # index 1 is the index of the maximum value
                    _, predicted = torch.max(outputs.data, 1)

                    # Number of labels
                    total += labels.size(0)

                    # Number of correct predictions
                    correct += (predicted == labels).sum()
                    
                    # Empty cache of GPU
                    del images
                    del labels
                    del loss
                    del outputs
                    torch.cuda.empty_cache()

                # Calculate accuracy
                accuracy = 100 * correct / total

                # Save parameters if loss is smaller
                if total_loss < minLoss:
                    minLoss = total_loss
                    best_parameters = model.state_dict()

                # Print accuracy and loss
                print(f'Iteration {iteration}: Loss: {total_loss}, Validation accuracy: {accuracy}')
    return best_parameters

In [8]:
import matplotlib.pyplot as plt
# Testing
def test(model):
    correct = 0
    total = 0
    total_loss = 0
    count = 0

    # Test Loop
    for images, labels in test_dataloader:

        images, labels = images.to(device), labels.to(device)

        # Make prediction
        outputs = model(images)

        loss = criterion(outputs, labels)
        total_loss += float(loss)

        # Get predictions from the maximum value, index 0 is the maximum value, 
        # index 1 is the index of the maximum value
        _, predicted = torch.max(outputs.data, 1)

        # Number of labels
        total += labels.size(0)

        # Number of correct predictions
        correct += (predicted == labels).sum()
        incorrect = images[predicted != labels]
        
        # Show 5 wrong predicted example
        if count < 5 and (predicted == labels).sum() != labels.size(0):
            
            wrong = "dog" if predicted[predicted != labels][0] else "cat"
            actual = "dog" if labels[predicted != labels][0] else "cat"
            print(f"Predicted {wrong}")
            print(f"Actual {actual}")
            show_image = incorrect[0].cpu().numpy().transpose((1, 2, 0))
            plt.imshow(show_image, cmap='gray')
            plt.show()
            count += 1
        
        # Empty cache
        del images
        del labels
        del loss
        del outputs
        torch.cuda.empty_cache()

    # Calculate accuracy
    accuracy = 100 * correct / total

    # Print accuracy and loss
    print(f'Best model has Loss: {total_loss}, Test accuracy: {accuracy}')

In [16]:
alexnet = models.alexnet(pretrained=True)
alexnet

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [17]:
alexnet.classifier[6] = nn.Linear(in_features=4096, out_features=2)

In [18]:
# Only unfreeze the last two layer
to_update = ["classifier.6.weight", "classifier.6.bias"]
to_update_params = []
for name, param in alexnet.named_parameters():
    if name in to_update:
        param.requires_grad = True
        to_update_params.append(param)
    else:
        param.requires_grad = False

In [19]:
# Training hyperparameters
epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=to_update_params, lr=0.01)

In [20]:
# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
alexnet = alexnet.to(device)

In [21]:
# Training
alexnet_best_parameters = training(alexnet)

Iteration 1: Loss: 39.91987943649292, Validation accuracy: 50.0


C:\Users\Leo\anaconda3\lib\site-packages\PIL\Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Iteration 10: Loss: 13.61647891998291, Validation accuracy: 72.5
Iteration 20: Loss: 12.186755061149597, Validation accuracy: 67.0
Iteration 30: Loss: 8.397380590438843, Validation accuracy: 75.33333587646484
Iteration 40: Loss: 10.339252710342407, Validation accuracy: 70.0
Iteration 50: Loss: 7.222352862358093, Validation accuracy: 73.16667175292969
Iteration 60: Loss: 7.052318453788757, Validation accuracy: 77.66667175292969
Iteration 70: Loss: 9.669580578804016, Validation accuracy: 78.0
Iteration 80: Loss: 8.248022675514221, Validation accuracy: 77.16667175292969
Iteration 90: Loss: 8.230790257453918, Validation accuracy: 74.33333587646484
Iteration 100: Loss: 7.5921560525894165, Validation accuracy: 76.66667175292969
Iteration 110: Loss: 7.402311682701111, Validation accuracy: 76.5
Iteration 120: Loss: 8.001210987567902, Validation accuracy: 75.16667175292969
Iteration 130: Loss: 6.787102103233337, Validation accuracy: 77.83333587646484
Iteration 140: Loss: 6.99583637714386, Valid

## GoogleNet

In [ ]:
googlenet = models.googlenet(pretrained=True)
googlenet

In [ ]:
googlenet.fc = nn.Linear(in_features=1024, out_features=3, bias=True)

In [ ]:
# Only unfreeze the last two layer
to_update = ["fc.weight", "fc.bias"]
to_update_params = []
for name, param in googlenet.named_parameters():
    if name in to_update:
        param.requires_grad = True
        to_update_params.append(param)
    else:
        param.requires_grad = False

In [ ]:
# Training hyperparameters
epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=to_update_params, lr=0.01)

In [ ]:
# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
googlenet = googlenet.to(device)

In [ ]:
# Training
googlenet_best_parameters = training(googlenet)

In [ ]:
# save the model params
torch.save(googlenet_best_parameters, ModelPath+'/googlenet_params.pt')

In [ ]:
googlenet.load_state_dict(torch.load("./best_models/googlenet_params.pt"))

In [ ]:
test(googlenet)

### Ensemble model

In [9]:
from torchensemble import VotingClassifier 

In [ ]:
ensemble_vote = VotingClassifier(estimator=alexnet, n_estimators=10)
criterion = nn.CrossEntropyLoss()
ensemble_vote.set_criterion(criterion)
ensemble_vote.set_optimizer("Adam", lr=0.01)
ensemble_vote.fit(train_dataloader, epochs=10)

In [ ]:
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = 100, shuffle = True)
test(ensemble)

In [10]:
# ensemble different models
class diff_vote_classifier():
    def __init__(self, *args):
        self.models = [model for model in args]
        self.n = len(self.models)
        
    def __call__(images):
        output = self.models[0](images) 
        for ind in range(1,len(self.models)):
            model = self.models[ind]
            output += model(images) # torch.add
        output = output/float(self.n)
        return output

In [ ]:
diff_voter = diff_vote_classifier(alexnet, googlenet)

In [ ]:
test(diff_voter)